In [115]:
import pandas as pd
import csv


#1 Carga y limpieza de datos


In [116]:
#Archivo a trabajar
file = 'EEG_eyes_classification.csv'
file_filtered = 'filtered_file.csv'
"""
Informacion del archivo:
-Class: b'1' = 0; b'2' = 1 y valores atipicos
Manejar valores nulos, identificar cada una de las columnas y corregir lo que se encuentre.
HINT: calcular el porcentaje de valores por columna es de gran ayuda, sin tener miedo a eliminar lo que no sirve.
"""

def calcular_porcentaje_vacios(fila):
    # Calcula el número de valores vacíos en la fila
    num_vacios = sum(1 for valor in fila if not valor.strip())
    # Calcula el porcentaje de valores vacíos
    porcentaje_vacios = (num_vacios / len(fila))
    return porcentaje_vacios

#Se filtran las columnas con mas de un dato faltante
with open(file, 'r') as csvfile, open(file_filtered, 'w', newline='') as newcsv:
    reader = csv.reader(csvfile, delimiter=',')
    writer = csv.writer(newcsv)
    for row in reader:
        porcentaje_vacio = calcular_porcentaje_vacios(row)

        if porcentaje_vacio < 1/16:#Antes 2/16
            if row[-1] == "b'1'":
                row[-1] = 0
            elif row[-1] == "b'2'":
                row[-1] = 1
            elif row[-1] == "Open":
                row[-1] = 0
            if row[-1] == "Close" or row[-1] == "Closed":
                row[-1] = 1
            writer.writerow(row)

#2 Balanceo de datos


In [117]:
df = pd.read_csv('filtered_file.csv')

count_0 = (df['Class'] == 0).sum()
count_1 = (df['Class'] == 1).sum()

diff = abs(count_0-count_1)

if count_0>count_1:
    df = df.drop(df[df['Class'] == 0].sample(n=diff).index)

else:
    df = df.drop(df[df['Class'] == 1].sample(n=diff).index)

count_0 = (df['Class'] == 0).sum()
count_1 = (df['Class'] == 1).sum()

df.to_csv('filtered_file.csv', index=False)

#3 Estandarización de datos

In [118]:
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('filtered_file.csv')

#Se seleccionan todas las columnas menos la ultima
columnas_a_estandarizar = df.columns[:-1]

scaler = StandardScaler()
#Se hace el estandarizado
df[columnas_a_estandarizar] = scaler.fit_transform(df[columnas_a_estandarizar])

df.to_csv('filtered_file.csv', index=False)

#4 Separación del data set:
60% Train
20% Test
20% Validation

In [119]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('filtered_file.csv')

X = df.drop(columns = ['Class'])
y = df['Class']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)

X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

#Se tiene X_train, y_train para TRAIN, X_test, y_test para el Testeo, y X_val, y_val para la Validacion

#5 Entrenamiento


In [120]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#Entrenamiento con hiperparametros default

#Random Forest
rf_default = RandomForestClassifier(random_state=42)
rf_default.fit(X_train, y_train)

#Desicion tree
dt_default = DecisionTreeClassifier(random_state=42)
dt_default.fit(X_train, y_train)

#Logistic regresion
lr_default = LogisticRegression(random_state=42)
lr_default.fit(X_train, y_train)

#Realizar la prediccion para el conjunto de Testeo
rf_pred_default = rf_default.predict(X_test)
dt_pred_default = dt_default.predict(X_test)
lr_pred_default = lr_default.predict(X_test)

#Calcular la precision de los modelos con hiperparametros default
rf_acc_default = accuracy_score(y_test, rf_pred_default)
dt_acc_default = accuracy_score(y_test, dt_pred_default)
lr_acc_default = accuracy_score(y_test, lr_pred_default)


print("Precisión con hiperparámetros predeterminados:")
print("Random Forest:", rf_acc_default)
print("Decision Tree:", dt_acc_default)
print("Regresión Logística:", lr_acc_default)

Precisión con hiperparámetros predeterminados:
Random Forest: 0.7748344370860927
Decision Tree: 0.6754966887417219
Regresión Logística: 0.5231788079470199
